In [353]:
from Bio.Seq import Seq
import vcf

In [354]:
with open('SARS-COV2.fasta') as f:
    file = f.read().replace('\n', '')
    

reference = file[2:]
spike_protein = reference[21562 : 25384]
amino_acid_seq_of_spike_protein = Seq(spike_protein)
amino_acid_seq_of_spike_protein = amino_acid_seq_of_spike_protein.translate()
with open('amino_acid_sequence_of_spike_protein.txt', 'w') as f:
    f.write(str(amino_acid_seq_of_spike_protein))

In [355]:
vcf_reader = vcf.Reader(open('var.vcf'))
variants = []
for record in vcf_reader:
    if record.POS >= 21562 and record.POS < 25384:
        variants.append((record.POS - 21562 , record.REF, record.ALT))
        
with open('variants.txt', 'w') as f:
    for var in variants:
        f.write(f'Position: {var[0]} Reference: {var[1]} Alt: {var[2][0]}\n')
                                                                 

In [356]:
functional = {"D":"acidic", "E": "acidic", "G": "polar",
"F":"nonpolar", "L": "nonpolar", "S": "polar", "Y": "polar",
"C": "polar", "W":"nonpolar", "L":"nonpolar", "P": "nonpolar",
"H": "basic", "Q":"polar", "R": "basic", "I":"nonpolar",
"M": "nonpolar", "T": "polar", "N": "polar", "K":"basic",
"S":"polar", "R":"basic", "V":"nonpolar","A":"nonpolar"}

In [357]:
mutation_output = []
variant_sequences = []

for i, var in enumerate(variants):
    pos, ref, alt = var
    mutation = list(reference[21562 : 25385])
    mutation[pos: pos + len(ref)] = list(str(alt[0]))
    mutation = ''.join(mutation)
    variant_sequences.append(mutation)
    amino_acid_mutation = Seq(mutation)
    amino_acid_mutation = amino_acid_mutation.translate()
    
    for j in range(min(len(amino_acid_mutation), len(amino_acid_seq_of_spike_protein))):
        if amino_acid_mutation[j] != amino_acid_seq_of_spike_protein[j]:
            if amino_acid_mutation[j] == '*':
                mutation_output.append('non-sense\n')
            else:
                mutation_output.append(str((f'non-synonomous: {functional[amino_acid_mutation[j]]} -> {functional[amino_acid_seq_of_spike_protein[j]]}\n')))
            break
    else:
        mutation_output.append('synonomous\n')
        
    
with open('variant_sequences.txt', 'w') as f:
    for var in variant_sequences:
        f.write(var + '\n')
with open('mutation_analysis.txt', 'w') as f:
    for m in mutation_output:
        f.write(m)
                                                                 
    
    